In [17]:
import pandas as pd
import numpy as np
dt = pd.read_pickle('Binning.pickle')
dt

,Education,JoiningYear,City,PaymentTier,Age,Gender,EverBenched,ExperienceInCurrentDomain,LeaveOrNot,JOININGYEAR_BIN,AGE_BIN,EXPERIENCEINCURRENTDOMAIN_BIN
1,Bachelors,2013,C,1,28,Female,No,3,1,"(2011.999, 2013.0]","(27.0, 28.0]","(2.0, 3.0]"
2,Bachelors,2014,B,3,38,Female,No,2,0,"(2013.0, 2014.0]","(34.0, 38.0]","(1.0, 2.0]"
3,Masters,2016,A,3,27,Male,No,5,1,"(2015.0, 2016.0]","(26.0, 27.0]","(4.0, 5.0]"
5,Bachelors,2016,A,3,22,Male,No,0,0,"(2015.0, 2016.0]","(21.999, 24.7]","(-0.001, 1.0]"
6,Bachelors,2015,B,3,38,Male,No,0,0,"(2014.0, 2015.0]","(34.0, 38.0]","(-0.001, 1.0]"
...,...,...,...,...,...,...,...,...,...,...,...,...
4647,Bachelors,2016,C,3,30,Male,No,2,0,"(2015.0, 2016.0]","(28.0, 31.0]","(1.0, 2.0]"
4648,Bachelors,2013,A,3,26,Female,No,4,0,"(2011.999, 2013.0]","(24.7, 26.0]","(3.0, 4.0]"
4649,Masters,2013,C,2,37,Male,No,2,1,"(2011.999, 2013.0]","(34.0, 38.0]","(1.0, 2.0]"
4650,Masters,2018,B,3,27,Male,No,5,1,"(2017.0, 2018.0]","(26.0, 27.0]","(4.0, 5.0]"


In [18]:
def num_event(event, dataframe ,field, val):
    if event == 'good':
        event = 1
    else:
        event = 0
    cond_main = dataframe[field] == val
    cond_event = dataframe['LeaveOrNot'] == event
    n = len(dataframe[cond_main & cond_event].index)
    return n


def dist_event(event, dataframe ,field, val):
    n = num_event(event, dataframe, field, val)
    sumd = 0
    all_elements = set(dataframe[field].to_numpy())
    for a in all_elements:
        s = num_event(event, dataframe, field, a)
        sumd = sumd + s
    return n/sumd

def WOE(dataframe, field, val):
    dg = dist_event('good', dataframe, field, val)
    db = dist_event('bad', dataframe, field, val)
    w = np.log(dg / db)
    return w

def IV(dataframe, field):
    elements = set(dataframe[field].to_numpy())
    n = len(elements)
    sumiv = 0
    for i in elements:
        dg = dist_event('good', dataframe, field, i)
        db = dist_event('bad', dataframe, field, i)
        s = (dg-db) * WOE(dataframe, field, i)
        sumiv = sumiv + s
    return sumiv   

fields = ['Education', 'PaymentTier', 'AGE_BIN', 'Gender', 'EverBenched','JOININGYEAR_BIN','EXPERIENCEINCURRENTDOMAIN_BIN']
list_woe=[]
list_attr=[]
list_val=[]
list_ngood = []
list_nbad = []
list_dgood = []
list_dbad = []

for f in fields:
    the_set = set(dt[f].to_numpy())
   
    for s in the_set:
        n_good = num_event('good', dt, f, s)
        n_bad = num_event('bad', dt, f, s)
        d_good = dist_event('good', dt, f, s)
        d_bad = dist_event('bad', dt, f, s)
        woe = WOE(dt, f, s)

        if 'BIN' in f:
            s = str(s)
        list_attr.append(f)
        list_val.append(s)
        list_ngood.append(n_good)
        list_nbad.append(n_bad)
        list_dgood.append(d_good)
        list_dbad.append(d_bad)
        list_woe.append(woe)



x = list(zip(list_attr, list_val, list_ngood, list_nbad, list_dgood, list_dbad,list_woe)) 
xlabel = ['ATTR', 'VAL', 'N_GOOD', 'N_BAD', 'DIST_GOOD', 'DIST_BAD','WOE']
df = pd.DataFrame(x, columns = xlabel )

df = df.sort_values(by=['ATTR', 'VAL'])

df = df.reset_index(drop=True)



df

,ATTR,VAL,N_GOOD,N_BAD,DIST_GOOD,DIST_BAD,WOE
0,AGE_BIN,"(21.999, 24.7]",127,199,0.114518,0.092601,0.212426
1,AGE_BIN,"(24.7, 26.0]",285,476,0.256988,0.221498,0.148615
2,AGE_BIN,"(26.0, 27.0]",157,265,0.141569,0.123313,0.138060
3,AGE_BIN,"(27.0, 28.0]",134,315,0.120830,0.146580,-0.193189
4,AGE_BIN,"(28.0, 31.0]",126,279,0.113616,0.129828,-0.133386
5,AGE_BIN,"(31.0, 34.0]",83,180,0.074842,0.083760,-0.112572
6,AGE_BIN,"(34.0, 38.0]",122,269,0.110009,0.125174,-0.129146
7,AGE_BIN,"(38.0, 41.0]",75,166,0.067628,0.077245,-0.132956
8,EXPERIENCEINCURRENTDOMAIN_BIN,"(-0.001, 1.0]",208,426,0.187556,0.198232,-0.055357
9,EXPERIENCEINCURRENTDOMAIN_BIN,"(1.0, 2.0]",277,488,0.249775,0.227082,0.095246


In [19]:
df.to_pickle('attr.pickle')

In [20]:
df_iv = pd.DataFrame({'ATTR': [], 'IV': []})
for f in fields:
    d = pd.DataFrame({'ATTR':[f], 'IV': [IV(dt, f)]})
    df_iv = df_iv.append(d, ignore_index = True)
df_iv

,ATTR,IV
0,Education,0.115893
1,PaymentTier,0.281629
2,AGE_BIN,0.023829
3,Gender,0.192766
4,EverBenched,0.038069
5,JOININGYEAR_BIN,1.205423
6,EXPERIENCEINCURRENTDOMAIN_BIN,0.018671
